# CPP

In [1]:
%%bash
cat > main.cpp << 'EOF'
#include <armadillo>
#include <iostream>
#include <chrono>

using namespace arma;
using namespace std;
using namespace std::chrono;

int main() {
    // Create a simple dataset:
    // X: a 5x1 matrix representing [1, 2, 3, 4, 5]
    // y: a 5x1 vector representing [2, 4, 6, 8, 10] (i.e. y = 2 * x)
    mat X(5, 1);
    X(0, 0) = 1.0;
    X(1, 0) = 2.0;
    X(2, 0) = 3.0;
    X(3, 0) = 4.0;
    X(4, 0) = 5.0;

    vec y(5);
    y(0) = 2.0;
    y(1) = 4.0;
    y(2) = 6.0;
    y(3) = 8.0;
    y(4) = 10.0;

    // Augment X with a column of ones for the intercept
    mat X_aug = join_horiz(ones<vec>(X.n_rows), X);

    // Time the training (solving for theta)
    auto train_start = high_resolution_clock::now();
    // Solve the normal equation: theta = (X_aug^T * X_aug)^-1 * (X_aug^T * y)
    vec theta = solve(X_aug, y);
    auto train_end = high_resolution_clock::now();
    duration<double, milli> train_duration = train_end - train_start;

    cout << "Learned parameters (theta): " << theta.t() << endl;
    cout << "Training time: " << train_duration.count() << " ms" << endl;

    // Predict for a new data point: x = 6
    mat X_new(1, 1);
    X_new(0, 0) = 6.0;
    mat X_new_aug = join_horiz(ones<vec>(X_new.n_rows), X_new);

    auto pred_start = high_resolution_clock::now();
    vec y_pred = X_new_aug * theta;
    auto pred_end = high_resolution_clock::now();
    duration<double, milli> pred_duration = pred_end - pred_start;

    cout << "Prediction for x = 6: " << y_pred[0] << endl;
    cout << "Prediction time: " << pred_duration.count() << " ms" << endl;

    return 0;
}
EOF

In [2]:
%%bash
g++ -O2 -std=c++11 main.cpp -o linear_regression -larmadillo -llapack -lblas

In [3]:
# %%bash
# g++ -O2 -std=c++11 main.cpp -o linear_regression -larmadillo -lopenblas

In [4]:
!./linear_regression

Learned parameters (theta):   -2.3832e-15   2.0000e+00

Training time: 18.9401 ms
Prediction for x = 6: 12
Prediction time: 0.008541 ms


# Rust

In [5]:
%%bash
curl https://sh.rustup.rs -sSf | sh -s -- -y
export PATH="$HOME/.cargo/bin:$PATH"
rustc --version


  stable-x86_64-unknown-linux-gnu installed - rustc 1.84.1 (e71f9a9a9 2025-01-27)


Rust is installed now. Great!

To get started you may need to restart your current shell.
This would reload your PATH environment variable to include
Cargo's bin directory ($HOME/.cargo/bin).

To configure your current shell, you need to source
the corresponding env file under $HOME/.cargo.

This is usually done by running one of the following (note the leading DOT):
. "$HOME/.cargo/env"            # For sh/bash/zsh/ash/dash/pdksh
source "$HOME/.cargo/env.fish"  # For fish
rustc 1.84.1 (e71f9a9a9 2025-01-27)


info: downloading installer
info: profile set to 'default'
info: default host triple is x86_64-unknown-linux-gnu
info: syncing channel updates for 'stable-x86_64-unknown-linux-gnu'
info: latest update on 2025-01-30, rust version 1.84.1 (e71f9a9a9 2025-01-27)
info: downloading component 'cargo'
info: downloading component 'clippy'
info: downloading component 'rust-docs'
info: downloading component 'rust-std'
info: downloading component 'rustc'
info: downloading component 'rustfmt'
info: installing component 'cargo'
info: installing component 'clippy'
info: installing component 'rust-docs'
info: installing component 'rust-std'
info: installing component 'rustc'
info: installing component 'rustfmt'
info: default toolchain set to 'stable-x86_64-unknown-linux-gnu'


In [6]:
%%bash
rm -rf rust_linear_regression
$HOME/.cargo/bin/cargo init --bin rust_linear_regression

    Creating binary (application) package
note: see more `Cargo.toml` keys and their definitions at https://doc.rust-lang.org/cargo/reference/manifest.html


In [7]:
%%bash
cd rust_linear_regression
# Append the dependency only if not already present
grep -q 'nalgebra' Cargo.toml || echo 'nalgebra = "0.32.0"' >> Cargo.toml
cd ..

In [8]:
%%bash
mkdir -p rust_linear_regression/src
cat > rust_linear_regression/src/main.rs << 'EOF'
use nalgebra::{DMatrix, DVector};
use std::time::Instant;

fn main() {
    // Create a dataset:
    // x = [1, 2, 3, 4, 5]
    // y = [2, 4, 6, 8, 10] (i.e. y = 2 * x)
    let x_values = vec![1.0, 2.0, 3.0, 4.0, 5.0];
    let y_values = vec![2.0, 4.0, 6.0, 8.0, 10.0];
    let n = x_values.len();

    // Build an augmented matrix (x_aug) directly:
    // The first column is ones (for the intercept), and the second column is x_values
    let x_aug = DMatrix::from_fn(n, 2, |i, j| if j == 0 { 1.0 } else { x_values[i] });

    // Build y as an n x 1 vector
    let y = DVector::from_vec(y_values);

    // Time the training (solving for theta)
    let train_start = Instant::now();
    let xt = x_aug.transpose();
    let xtx = &xt * &x_aug;
    let xty = &xt * &y;
    let theta = xtx.try_inverse().expect("Matrix is invertible") * xty;
    let train_duration = train_start.elapsed();

    // Print theta with 4 decimal places per element
    print!("Learned parameters (theta): [");
    for i in 0..theta.len() {
        if i > 0 {
            print!(", ");
        }
        // Print each element with 4 decimal places
        print!("{:.4}", theta[i]);
    }
    println!("]");
    println!("Training time: {:.4} ms", train_duration.as_secs_f64() * 1000.0);

    // For prediction, create an augmented matrix for x = 6: [1, 6]
    let x_new_aug = DMatrix::from_fn(1, 2, |_, j| if j == 0 { 1.0 } else { 6.0 });
    let pred_start = Instant::now();
    let y_pred = &x_new_aug * &theta;
    let pred_duration = pred_start.elapsed();

    println!("Prediction for x = 6: {:.4}", y_pred[0]);
    println!("Prediction time: {:.4} ms", pred_duration.as_secs_f64() * 1000.0);
}
EOF

In [9]:
%%bash
cd rust_linear_regression
$HOME/.cargo/bin/cargo run --release --quiet
cd ..

Learned parameters (theta): [0.0000, 2.0000]
Training time: 0.0006 ms
Prediction for x = 6: 12.0000
Prediction time: 0.0003 ms


In [10]:
# %%bash
# cd rust_linear_regression
# $HOME/.cargo/bin/cargo build --release --quiet
# ./target/release/rust_linear_regression
# cd ..

# GoLang

In [11]:
%%bash
sudo apt-get update
sudo apt-get install -y golang
go version

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,309 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,660 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,230 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,523 kB]
Hit:13 https://ppa.la

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 8.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


In [12]:
%%bash
mkdir -p golang_linreg
cd golang_linreg
# Initialize the module if not already created
go mod init example.com/linreg || echo "Module already exists"
# Get Gonum v0.14.0 explicitly
go get gonum.org/v1/gonum/mat@v0.14.0
# Force the entire gonum.org/v1/gonum module to version v0.14.0
go mod edit -replace=gonum.org/v1/gonum=gonum.org/v1/gonum@v0.14.0
# Tidy up the module
go mod tidy
cd ..

go: creating new go.mod: module example.com/linreg
go: downloading gonum.org/v1/gonum v0.14.0
go: added gonum.org/v1/gonum v0.14.0
go: warning: "all" matched no packages


In [13]:
%%bash
cat > golang_linreg/main.go << 'EOF'
package main

import (
	"fmt"
	"math"
	"time"

	"gonum.org/v1/gonum/mat"
)

// normalizeZero returns 0.0 if x is very close to zero
func normalizeZero(x float64, tol float64) float64 {
	if math.Abs(x) < tol {
		return 0.0
	}
	return x
}

func main() {
	// Create a dataset:
	// x = [1, 2, 3, 4, 5]
	// y = [2, 4, 6, 8, 10] (i.e. y = 2 * x)
	xValues := []float64{1, 2, 3, 4, 5}
	yValues := []float64{2, 4, 6, 8, 10}
	n := len(xValues)

	// Augment data: create an n x 2 matrix (first column ones, second column xValues)
	XAug := mat.NewDense(n, 2, nil)
	for i := 0; i < n; i++ {
		XAug.Set(i, 0, 1.0)
		XAug.Set(i, 1, xValues[i])
	}

	// Build y as an n x 1 vector
	y := mat.NewVecDense(n, yValues)

	// Time the training (solving for theta)
	trainStart := time.Now()
	var Xt mat.Dense
	Xt.Mul(XAug.T(), XAug) // Xt = XAug^T * XAug

	var XtInv mat.Dense
	if err := XtInv.Inverse(&Xt); err != nil {
		fmt.Println("Matrix inversion failed:", err)
		return
	}

	var Xty mat.VecDense
	Xty.MulVec(XAug.T(), y)

	var theta mat.VecDense
	theta.MulVec(&XtInv, &Xty)
	trainDuration := time.Since(trainStart)

	// Normalize theta values (so that very-small numbers print as 0.0000)
	tol := 1e-12
	data := theta.RawVector().Data
	for i, v := range data {
		data[i] = normalizeZero(v, tol)
	}

	fmt.Printf("Learned parameters (theta): [")
	for i, v := range data {
		if i > 0 {
			fmt.Printf(", ")
		}
		fmt.Printf("%.4f", v)
	}
	fmt.Println("]")

	// Print training time in both milliseconds and microseconds
	trainMillis := float64(trainDuration.Milliseconds())
	trainMicros := float64(trainDuration.Microseconds())
	fmt.Printf("Training time: %.6f ms\n", trainMillis)
	fmt.Printf("Training time: %.6f µs\n", trainMicros)

	// Predict for a new data point: x = 6
	// Create an augmented row for x = 6: [1, 6]
	XNewAug := mat.NewDense(1, 2, []float64{1.0, 6.0})

	predStart := time.Now()
	var yPred mat.Dense
	yPred.Mul(XNewAug, &theta) // yPred is a 1 x 1 matrix
	predDuration := time.Since(predStart)

	predMillis := float64(predDuration.Milliseconds())
	predMicros := float64(predDuration.Microseconds())
	fmt.Printf("Prediction for x = 6: %.4f\n", yPred.At(0, 0))
	fmt.Printf("Prediction time: %.6f ms\n", predMillis)
	fmt.Printf("Prediction time: %.6f µs\n", predMicros)
}
EOF

In [14]:
%%bash
cd golang_linreg
export GO111MODULE=on
go mod tidy && go mod download
go run .
cd ..

Learned parameters (theta): [0.0000, 2.0000]
Training time: 0.000000 ms
Training time: 152.000000 µs
Prediction for x = 6: 12.0000
Prediction time: 0.000000 ms
Prediction time: 1.000000 µs


go: found gonum.org/v1/gonum/mat in gonum.org/v1/gonum v0.0.0-00010101000000-000000000000
go: downloading golang.org/x/exp v0.0.0-20230321023759-10a507213a29
